# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [1]:
!pip install selenium

In [2]:
# Passo 1 Pegar cotação do dolar
# Passo 2 Pegar cotação do ouro ...
# Passo 3 Pegar cotação do euro
# Passo 4 importar base de produtos e atualizar os preços da base
# Passo 5 recalcular os preços
# Passo 6 exportar a base de dados existentes

In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys


#***para nao aparecer o navegador rodando em segundo plano***
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.headless = True
navegador = webdriver.Chrome(options=chrome_options)

In [11]:

# Passo 1 Pegar cotação do dolar
# entrar no google
# pesquisar cotação do dolar

navegador = webdriver.Chrome()
navegador.get('https://www.google.com/')

navegador.find_element("xpath",
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dolar")

navegador.find_element('xpath',
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')

navegador.get('https://www.google.com/')

navegador.find_element("xpath",
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")

navegador.find_element('xpath',
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element('xpath','//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')



navegador.get('https://www.melhorcambio.com/ouro-hoje')

cotacao_ouro = navegador.find_element('xpath','//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',','.')
print(cotacao_dolar)
print(cotacao_euro)
print(cotacao_ouro)

#fechando o navegador
navegador.quit()


5.108899999999999
5.5389416570000005
312.26


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [12]:
# Passo 4 importar base de produtos e atualizar os preços da base
import pandas as pd

dadostabela = pd.read_excel("Produtos.xlsx")
display(dadostabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [13]:
# Passo 5 recalcular os preços
# atualizar a cotação
# recalcular os preços
# -------------------------------------------
# preços de compra = cotação * preço original
# preços de venda = preços de compra * margem

#atualizando cotacao dolar

# dois iguais assim é comparação se a coluna moeda é igual a dólar
dadostabela.loc[dadostabela['Moeda'] == "Dólar","Cotação"] = float(cotacao_dolar)
dadostabela.loc[dadostabela['Moeda'] == "Euro","Cotação"] = float(cotacao_euro)
dadostabela.loc[dadostabela['Moeda'] == "Ouro","Cotação"] = float(cotacao_ouro)

display(dadostabela)


,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.108900,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,5.538942,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5.108900,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5.108900,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,5.538942,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5.108900,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,312.260000,7000.00,1.15,8050.000


In [15]:
# recalcular os preços
dadostabela['Preço de Compra'] = dadostabela['Cotação']*dadostabela['Preço Original']
dadostabela['Preço de Venda'] = dadostabela['Preço de Compra']*dadostabela['Margem']

display(dadostabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.108900,5108.848911,1.40,7152.388475
1,Carro Renault,4500.00,Euro,5.538942,24925.237457,2.00,49850.474913
2,Notebook Dell,899.99,Dólar,5.108900,4597.958911,1.70,7816.530149
3,IPhone,799.00,Dólar,5.108900,4082.011100,1.70,6939.418870
4,Carro Fiat,3000.00,Euro,5.538942,16616.824971,1.90,31571.967445
5,Celular Xiaomi,480.48,Dólar,5.108900,2454.724272,2.00,4909.448544
6,Joia 20g,20.00,Ouro,312.260000,6245.200000,1.15,7181.980000


### Agora vamos exportar a nova base de preços atualizada

In [18]:
# Passo 6 exportar a base de dados existentes
dadostabela.to_excel("Produtosatualizados.xlsx",index=False)